# GPM constellation

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from datetime import datetime

In [2]:
start_date = datetime(2022, 9, 28, 10, 0)
end_date = datetime(2022, 9, 28, 14, 0)

In [3]:
from datetime import datetime, timedelta
from pansat.products.satellite.gpm import l2a_dpr
l2a_dpr.download(start_date, end_date, "/home/simonpf/data/gprof_nn/constellation/dpr")

Please enter your pansat user password:
········


[PosixPath('/home/simonpf/data/gprof_nn/constellation/dpr/2A.GPM.DPR.V9-20211125.20220928-S093055-E110326.048766.V07A.HDF5'),
 PosixPath('/home/simonpf/data/gprof_nn/constellation/dpr/2A.GPM.DPR.V9-20211125.20220928-S110327-E123559.048767.V07A.HDF5'),
 PosixPath('/home/simonpf/data/gprof_nn/constellation/dpr/2A.GPM.DPR.V9-20211125.20220928-S123600-E140831.048768.V07A.HDF5')]

In [3]:
VARS = ["longitude", "latitude", "surface_precip", "scan_time"]

def load_precip_data(path):
    files = sorted(list(path.glob("*.nc")))
    data = [xr.open_dataset(filename)[VARS] for filename in files]
    data = xr.concat(data, dim="scans")
    data.surface_precip.data = np.maximum(data.surface_precip.data, 1e-3)
    return data

def load_obs_data(product, path, subsample=None):
    files = sorted(list(path.glob("*.HDF5")))
    if subsample is not None:
        data = [product.open(filename) for filename in files]
    else:
        data = [product.open(filename)[{"scans": slice(0, None, subsample)}] for filename in files]
    data = xr.concat(data, dim="scans")
    if "latitude_s1" in data:
        data = data.rename({
            "latitude_s1": "latitude",
            "longitude_s1": "longitude"
        })
    return data


In [5]:
from pansat.products.satellite.gpm import l2a_dpr
from pansat.products.satellite.gpm import (
    l1c_npp_atms,
    l1c_metopc_mhs,
    l1c_metopb_mhs,
    l1c_gpm_gmi_r,
    l1c_gcomw1,
    l1c_noaa19_mhs,
    l1c_noaa20_atms,
    l1c_f17_ssmis,
    l1c_f18_ssmis,
    l1c_noaa20_atms
)

data_path = Path("/home/simonpf/data/gprof_nn/")
npp_data = load_precip_data(data_path / "constellation"/ "npp")
metopb_data = load_precip_data(data_path / "constellation"/ "metop_b")
metopc_data = load_precip_data(data_path / "constellation"/ "metop_c")
noaa19_data = load_precip_data(data_path / "constellation" / "noaa19")
gmi_data = load_precip_data(data_path / "constellation"/ "gmi")
amsr2_data = load_precip_data(data_path / "constellation"/ "amsr2")
f16_data = load_precip_data(data_path / "constellation"/ "f16")
f17_data = load_precip_data(data_path / "constellation"/ "f17")
f18_data = load_precip_data(data_path / "constellation"/ "f18")
noaa20_data = load_precip_data(data_path / "constellation"/ "noaa20")

data_path = Path("/home/simonpf/data/gprof_nn/")
npp_sat_data = load_obs_data(l1c_npp_atms, data_path / "obs"/ "npp")
metopb_sat_data = load_obs_data(l1c_metopb_mhs, data_path / "obs"/ "metop_b")
metopc_sat_data = load_obs_data(l1c_metopc_mhs, data_path / "obs"/ "metop_c")
gmi_sat_data = load_obs_data(l1c_gpm_gmi_r, data_path / "obs"/ "gmi")
amsr2_sat_data = load_obs_data(l1c_gcomw1, data_path / "obs"/ "gcomw1")
noaa19_sat_data = load_obs_data(l1c_noaa19_mhs, data_path / "obs" / "noaa19")
f16_sat_data = load_obs_data(l1c_f17_ssmis, data_path / "obs" / "f16")
f17_sat_data = load_obs_data(l1c_f17_ssmis, data_path / "obs" / "f17")
f18_sat_data = load_obs_data(l1c_f18_ssmis, data_path / "obs" / "f18")
noaa20_sat_data = load_obs_data(l1c_noaa20_atms, data_path / "obs" / "noaa20")
#dpr_sat_data = load_obs_data(l2a_dpr, data_path / "constellation" /  "dpr", subsample=10)

In [13]:
from satviewer import create_texture_area
area = create_texture_area(360 / 2700)

## Surface

In [12]:
import pyvista as pv
from PIL import Image
from pyresample.kd_tree import resample_nearest
from satviewer import create_texture_area

background = np.array(Image.open("/home/simonpf/data/natural_earth/blue_marble_sep.jpg"))
background = background[::2, ::2]
earth_sfc = pv.numpy_to_texture(background)

In [12]:
from matplotlib.colors import LogNorm, Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.cm import get_cmap
import seaborn as sns
sns.reset_orig()
norm = LogNorm(1e-1, 5e1)
cmap = get_cmap("mako_r")
cmap.set_bad(np.array([1.0, 0.0, 0.0, 0.0]))
mappable = ScalarMappable(norm=norm, cmap=cmap)
tb_mappable = ScalarMappable(norm=Normalize(150, 300), cmap=cmap)

/tmp/ipykernel_3222824/433732433.py:8: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("mako_r").copy()
  cmap.set_bad(np.array([1.0, 0.0, 0.0, 0.0]))


In [13]:
gmi_sat_data["tbs_89"] = (("scans", "pixels"), gmi_sat_data["tbs_s1"].data[:, :, 7])
gmi_data["tbs_89"] = (("scans", "pixels"), gmi_sat_data["tbs_s1"].data[:, :, 7])

In [14]:
from satviewer import render_swath_surface, render_swath
import pyvista as pv

surf = render_swath_surface(
    gmi_sat_data,
    "tbs_89",
    (...),
    np.datetime64("2022-09-28T00:00:00"),
    mappable=mappable,
    use_elevation=False
)
txt = render_swath(
    gmi_data,
    "surface_precip",
    (...),
    np.datetime64("2022-09-28T00:00:00"),
    mappable=mappable
)
t = pv.numpy_to_texture(np.array(txt))
surf.textures["precip"] = t

In [15]:
surf.plot(opacity=1.0)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## GMI + DPR

In [16]:
from matplotlib.colors import LogNorm, Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.cm import get_cmap
import seaborn as sns
sns.reset_orig()

cmap = get_cmap("mako_r")
cmap.set_bad(np.array([1.0, 0.0, 0.0, 0.0]))
tb_mappable = ScalarMappable(norm=Normalize(150, 300), cmap=cmap)

/tmp/ipykernel_3222824/2074671630.py:8: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("mako_r").copy()
  cmap.set_bad(np.array([1.0, 0.0, 0.0, 0.0]))


In [14]:
from satviewer import create_earth
earth = create_earth(area)
earth.textures["surface"] = earth_sfc

In [18]:
time = np.datetime64("2022-09-28T10:30:00"),

In [20]:
dpr_sat_data["latitude"] = dpr_sat_data["latitude_hs"]
dpr_sat_data["longitude"] = dpr_sat_data["longitude_hs"]

In [21]:
from satviewer import render_radar_swath
pv.set_plot_theme("document")
plot = pv.Plotter()
render_radar_swath(plot, dpr_sat_data, time)

6793766.0 2232793.2
72146


In [22]:
plot.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [34]:
from satviewer import R
from satviewer.satellites import LEOSat
gmi = LEOSat("GPM", gmi_sat_data)
#npp = LEOSat("Suomi NPP", npp_sat_data)
#f16 = LEOSat("F16", f16_sat_data)
#f17 = LEOSat("F17", f17_sat_data)
#f18 = LEOSat("F18", f18_sat_data)
#metopb = LEOSat("Metop-B", metopb_sat_data)
#metopc = LEOSat("Metop-C", metopc_sat_data)
#noaa19 = LEOSat("NOAA 19", noaa19_sat_data)
#noaa20 = LEOSat("NOAA 20", noaa20_sat_data)
#amsr2 = LEOSat("AMSR 2", amsr2_sat_data)

leo_sats = [
    (gmi, gmi_data),
    #(amsr2, amsr2_data),
    #(metopb, metopb_data),
    #(metopc, metopc_data),
    #(noaa19, noaa19_data),
    #(f16, f16_data),
    #(f17, f17_data),
    #(f18, f18_data),
    #(noaa20, noaa20_data),
    #(npp, npp_data)
]

In [68]:
import pyvista as pv
pv.set_plot_theme("document")
plot = pv.Plotter(
    multi_samples=32,
    polygon_smoothing=True,
    line_smoothing=True,
    window_size=(1024, 1024)
)
plot.add_mesh(earth)
time = np.datetime64("2022-09-28T13:10:00")
for sat, data in leo_sats:
    sat.add(plot, time)
render_radar_swath(plot, dpr_sat_data, time)


4594522.0 -6548379.0
244417


In [69]:
from satviewer import render_swath_surface, render_swath
from pansat.time import to_datetime
import pyvista as pv


def plot_time(plot, time):
    plot.clear()
    plot.add_mesh(earth)
    
    x = np.zeros((3, 2))
    y = np.zeros((3, 2))
    #dummy = pv.StructuredGrid(x, y, x + y )
    #dummy["Surface precip [mm/h]"] = [1e-2, 2e1]
    #plot.add_mesh(dummy, scalars="Surface precip [mm/h]", cmap="mako_r", log_scale=True)
    
    for sat, precip_data in leo_sats:
        sat.add(plot, time)
        
        surf = render_swath_surface(
            precip_data,
            "tbs_89",
            (...),
            time,
            mappable=tb_mappable,
            use_elevation=False
        )
        txt = render_swath(
            precip_data,
            "tbs_89",
            (...),
            time,
            mappable=tb_mappable
        )
        txt = pv.numpy_to_texture(np.array(txt))
        surf.textures["precip"] = txt
        plot.add_mesh(surf)
        
        time_str = to_datetime(time).strftime("%y-%m-%d %H:%M UTC")
        plot.add_title(time_str, font_size=15)
    render_radar_swath(plot, dpr_sat_data, time)


In [30]:
from satviewer.satellites import R
def set_camera(camera, lon, lat, z):
    x = z * np.cos(np.deg2rad(lat)) * np.cos(np.deg2rad(lon))
    y = z * np.cos(np.deg2rad(lat)) * np.sin(np.deg2rad(lon))
    z = z * np.sin(np.deg2rad(lat))
    camera.position = (x, y, z)
    

In [71]:
set_camera(plot.camera, 10, 20, 5.4 * R)

In [72]:
plot_time(plot, time)

4594522.0 -6548379.0
244417


In [73]:
time

numpy.datetime64('2022-09-28T13:10:00')

In [74]:
plot.show(auto_close=False)

ViewInteractiveWidget(height=1024, layout=Layout(height='auto', width='100%'), width=1024)

## Constellation

In [25]:
from matplotlib.colors import LogNorm, Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.cm import get_cmap
import seaborn as sns
sns.reset_orig()

cmap = get_cmap("mako_r")
cmap.set_bad(np.array([1.0, 0.0, 0.0, 0.0]))
mappable = ScalarMappable(norm=LogNorm(1e-1, 1e2), cmap=cmap)

/tmp/ipykernel_3224219/4164900508.py:8: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("mako_r").copy()
  cmap.set_bad(np.array([1.0, 0.0, 0.0, 0.0]))


In [26]:
from satviewer import R
from satviewer.satellites import LEOSat
gmi = LEOSat("GPM", gmi_sat_data)
npp = LEOSat("Suomi NPP", npp_sat_data)
f16 = LEOSat("F16", f16_sat_data)
f17 = LEOSat("F17", f17_sat_data)
f18 = LEOSat("F18", f18_sat_data)
metopb = LEOSat("Metop-B", metopb_sat_data)
metopc = LEOSat("Metop-C", metopc_sat_data)
noaa19 = LEOSat("NOAA 19", noaa19_sat_data)
noaa20 = LEOSat("NOAA 20", noaa20_sat_data)
amsr2 = LEOSat("AMSR 2", amsr2_sat_data)

leo_sats = [
    (gmi, gmi_data),
    (amsr2, amsr2_data),
    (metopb, metopb_data),
    (metopc, metopc_data),
    (noaa19, noaa19_data),
    (f16, f16_data),
    (f17, f17_data),
    (f18, f18_data),
    (noaa20, noaa20_data),
    (npp, npp_data)
]

In [27]:
from satviewer import render_swath_surface, render_swath
from pansat.time import to_datetime
import pyvista as pv


def plot_time(plot, time):
    plot.clear()
    plot.add_mesh(earth)
    
    x = np.zeros((3, 2))
    y = np.zeros((3, 2))
    dummy = pv.StructuredGrid(x, y, x + y )
    dummy["Surface precip [mm/h]"] = [1e-2, 2e1]
    plot.add_mesh(dummy, scalars="Surface precip [mm/h]", cmap="mako_r", log_scale=True)
    
    for sat, precip_data in leo_sats:
        sat.add(plot, time)
        
        surf = render_swath_surface(
            precip_data,
            "surface_precip",
            (...),
            time,
            mappable=mappable,
            use_elevation=False
        )
        txt = render_swath(
            precip_data,
            "surface_precip",
            (...),
            time,
            mappable=mappable
        )
        txt = pv.numpy_to_texture(np.array(txt))
        surf.textures["precip"] = txt
        plot.add_mesh(surf)
        
        time_str = to_datetime(time).strftime("%y-%m-%d %H:%M UTC")
        plot.add_title(time_str, font_size=15)

In [28]:
import pyvista as pv
pv.set_plot_theme("document")
plot = pv.Plotter(
    multi_samples=32,
    polygon_smoothing=True,
    line_smoothing=True,
    window_size=(1024, 1024)
)
plot.add_mesh(earth)
time = np.datetime64("2022-09-28T13:10:00")
for sat, data in leo_sats:
    sat.add(plot, time)

In [21]:
times = np.arange(
    np.datetime64("2022-09-28T12:40:00"),
    np.datetime64("2022-09-28T13:30:00"),
    np.timedelta64(1 * 10, "s"),
)

In [33]:
plot.open_movie("gpm_constellation.mp4")

In [34]:
set_camera(plot.camera, 10, 20, 5.4 * R)

In [35]:
for time in times:
    plot_time(plot, time)
    plot.write_frame()

In [36]:
plot.close()

In [83]:
metopb.sat.points

pyvista_ndarray([[ 1103802.40617544, -2097378.40660681,
                  -6818607.60589666]])

In [70]:
plot.update_coordinates?

In [54]:
goes.filename_to_date(files[0].name)

datetime.datetime(2022, 4, 11, 15, 20, 20)